In [1]:
# https://www.cnblogs.com/machangwei-8/p/15925358.html
from kubernetes import client, config
# 加载认证文件
# config.kube_config.load_kube_config("minikube_config.yml")
config.kube_config.load_kube_config("config.yml")

In [2]:
#获取CoreV1Api和BatchV1Api版本对象
core_v1 = client.CoreV1Api()  #用于操作核心v1 API组中的资源，如Pod、Service、Node、Namespace等
batch_v1 = client.BatchV1Api()  #用于操作Job等资源
apps_v1 = client.AppsV1Api()  #用于操作Deployment、DaemonSet、ReplicaSet和StatefulSet
custom_api = client.CustomObjectsApi()  #用于操作CRD

In [3]:
# 列出命名空间
for ns in core_v1.list_namespace().items:
    print(ns.metadata.name)

default
kube-node-lease
kube-public
kube-system
kuboard


In [4]:
# 列出所有nodes
for node in core_v1.list_node().items:
    name = node.metadata.name
    status = node.status.conditions[-1].type if node.status.conditions[-1].status == "True" else "NotReady"
    ip = node.status.addresses[0].address
    version = node.status.node_info.kubelet_version
    os = node.status.node_info.os_image
    print("name:{},status:{},ip:{},version:{},os:{}".format(name, status, ip,version,os))

name:bigdata1,status:Ready,ip:192.168.0.44,version:v1.17.4,os:CentOS Linux 7 (Core)
name:bigdata2,status:Ready,ip:192.168.0.18,version:v1.17.4,os:CentOS Linux 7 (Core)
name:bigdata3,status:Ready,ip:192.168.0.67,version:v1.17.4,os:CentOS Linux 7 (Core)
name:bigdata4,status:Ready,ip:192.168.0.95,version:v1.17.4,os:CentOS Linux 7 (Core)


In [5]:
# 列出所有service，对应命令：kubectl get service --all-namespaces -o wide
for service in core_v1.list_service_for_all_namespaces().items:
    print(service.metadata.name)
    print(service.spec.cluster_ip)
    print(service.metadata.namespace)
    print(service.spec.ports)
    print(service.spec.type)
    print(service.spec.selector)

kubernetes
10.96.0.1
default
[{'app_protocol': None,
 'name': 'https',
 'node_port': None,
 'port': 443,
 'protocol': 'TCP',
 'target_port': 6443}]
ClusterIP
None
kube-dns
10.96.0.10
kube-system
[{'app_protocol': None,
 'name': 'dns',
 'node_port': None,
 'port': 53,
 'protocol': 'UDP',
 'target_port': 53}, {'app_protocol': None,
 'name': 'dns-tcp',
 'node_port': None,
 'port': 53,
 'protocol': 'TCP',
 'target_port': 53}, {'app_protocol': None,
 'name': 'metrics',
 'node_port': None,
 'port': 9153,
 'protocol': 'TCP',
 'target_port': 9153}]
ClusterIP
{'k8s-app': 'kube-dns'}
kubelet
None
kube-system
[{'app_protocol': None,
 'name': 'https-metrics',
 'node_port': None,
 'port': 10250,
 'protocol': 'TCP',
 'target_port': 10250}, {'app_protocol': None,
 'name': 'http-metrics',
 'node_port': None,
 'port': 10255,
 'protocol': 'TCP',
 'target_port': 10255}, {'app_protocol': None,
 'name': 'cadvisor',
 'node_port': None,
 'port': 4194,
 'protocol': 'TCP',
 'target_port': 4194}]
ClusterIP
None

In [6]:
# 列出指定ns的pod
for pod in core_v1.list_namespaced_pod("default").items:
    print("name:{},ip:{},ns:{}".format(pod.metadata.name ,pod.status.pod_ip,pod.metadata.namespace))

In [7]:
# 列出所有ns的pod
for pod in core_v1.list_pod_for_all_namespaces().items:
    print("name:{},ip:{},ns:{}".format(pod.metadata.name ,pod.status.pod_ip,pod.metadata.namespace))

name:calico-kube-controllers-6b94766748-f55lq,ip:10.244.191.214,ns:kube-system
name:calico-node-5sdft,ip:192.168.0.95,ns:kube-system
name:calico-node-cpq7b,ip:192.168.0.67,ns:kube-system
name:calico-node-tfj95,ip:192.168.0.44,ns:kube-system
name:calico-node-vqb2x,ip:192.168.0.18,ns:kube-system
name:coredns-6955765f44-f28m2,ip:10.244.191.213,ns:kube-system
name:coredns-6955765f44-wcfvm,ip:10.244.251.9,ns:kube-system
name:etcd-bigdata1,ip:192.168.0.44,ns:kube-system
name:kube-apiserver-bigdata1,ip:192.168.0.44,ns:kube-system
name:kube-controller-manager-bigdata1,ip:192.168.0.44,ns:kube-system
name:kube-proxy-29p4r,ip:192.168.0.44,ns:kube-system
name:kube-proxy-sch6n,ip:192.168.0.18,ns:kube-system
name:kube-proxy-w62dp,ip:192.168.0.67,ns:kube-system
name:kube-proxy-x8469,ip:192.168.0.95,ns:kube-system
name:kube-scheduler-bigdata1,ip:192.168.0.44,ns:kube-system
name:metrics-server-5cf78b8dc6-g88pp,ip:10.244.34.138,ns:kube-system
name:metrics-server-5cf78b8dc6-gmljh,ip:10.244.251.12,ns:kube

In [21]:
# 查看crd资源
for cr in custom_api.list_namespaced_custom_object(group="stable.example.com",version="v1",namespace="default",plural="crontabs")["items"]:
    print(cr)

{'apiVersion': 'stable.example.com/v1', 'kind': 'CronTab', 'metadata': {'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"stable.example.com/v1","kind":"CronTab","metadata":{"annotations":{},"name":"my-new-cron-object","namespace":"default"},"spec":{"cronSpec":"* * * * */5","image":"my-awesome-cron-image","replicas":3}}\n'}, 'creationTimestamp': '2023-04-20T07:30:00Z', 'generation': 1, 'name': 'my-new-cron-object', 'namespace': 'default', 'resourceVersion': '1992820', 'selfLink': '/apis/stable.example.com/v1/namespaces/default/crontabs/my-new-cron-object', 'uid': 'c5479ffe-3d08-48b3-ba84-7ee56bd493af'}, 'spec': {'cronSpec': '* * * * */5', 'image': 'my-awesome-cron-image', 'replicas': 3}}
